<a href="https://colab.research.google.com/github/hasibzunair/learn-pytorch/blob/master/notebooks/intro/Tutorial5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tutorial 5
Adapted from https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html

In [1]:
%matplotlib inline


`Learn the Basics <intro.html>`_ ||
`Quickstart <quickstart_tutorial.html>`_ || 
`Tensors <tensorqs_tutorial.html>`_ ||
`Datasets & DataLoaders <data_tutorial.html>`_ ||
`Transforms <transforms_tutorial.html>`_ ||
**Build Model** ||
`Autograd <autogradqs_tutorial.html>`_ ||
`Optimization <optimization_tutorial.html>`_ ||
`Save & Load Model <saveloadrun_tutorial.html>`_

Build the Neural Network
===================

Neural networks comprise of layers/modules that perform operations on data. 
The `torch.nn <https://pytorch.org/docs/stable/nn.html>`_ namespace provides all the building blocks you need to 
build your own neural network. Every module in PyTorch subclasses the `nn.Module <https://pytorch.org/docs/stable/generated/torch.nn.Module.html>`_. 
A neural network is a module itself that consists of other modules (layers). This nested structure allows for
building and managing complex architectures easily.

In the following sections, we'll build a neural network to classify images in the FashionMNIST dataset.




In [2]:
import os
import torch
from torch import nn # (all building blocks to build NNs!)
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
-----------------------
We want to be able to train our model on a hardware accelerator like the GPU, 
if it is available. Let's check to see if 
`torch.cuda <https://pytorch.org/docs/stable/notes/cuda.html>`_ is available, else we 
continue to use the CPU. 



In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


Define the Class
-------------------------
We define our neural network by subclassing ``nn.Module``, and 
initialize the neural network layers in ``__init__``. Every ``nn.Module`` subclass implements
the operations on input data in the ``forward`` method. 



In [4]:
# The new class inherits from nn.Module

class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU() #10-dimensional tensor with raw values, not softmax
        )

    def forward(self, x):
        # operations applied to the input data are defined here
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of ``NeuralNetwork``, and move it to the ``device``, and print 
its structure.



In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


To use the model, we pass it the input data. This executes the model's ``forward``,
along with some `background operations <https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866>`_. 
Do not call ``model.forward()`` directly!

Calling the model on the input returns a 10-dimensional tensor with raw predicted values for each class.
We get the prediction probabilities by passing it through an instance of the ``nn.Softmax`` module.



In [7]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X) # raw values
pred_probab = nn.Softmax(dim=1)(logits) # apply sotfmax
y_pred = pred_probab.argmax(1) # similar to np.argmax
print(f"Predicted class: {y_pred}")

Predicted class: tensor([7], device='cuda:0')


--------------




Model Layers
-------------------------

Lets break down the layers in the FashionMNIST model. To illustrate it, we 
will take a sample minibatch of 3 images of size 28x28 and see what happens to it as 
we pass it through the network. 



In [8]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


nn.Flatten
^^^^^^^^^^^^^^^^^^^^^^
We initialize the `nn.Flatten  <https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html>`_ 
layer to convert each 2D 28x28 image into a contiguous array of 784 pixel values (
the minibatch dimension (at dim=0) is maintained).



In [9]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


nn.Linear 
^^^^^^^^^^^^^^^^^^^^^^
The `linear layer <https://pytorch.org/docs/stable/generated/torch.nn.Linear.html>`_
is a module that applies a linear transformation on the input using its stored weights and biases.




In [10]:
layer1 = nn.Linear(in_features=784, out_features=20) # y = Wx + b, W -> weights, x-> input data
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [11]:
hidden1[0]

tensor([-0.4893,  0.2240,  0.3864,  0.4155,  0.2082, -0.3094, -0.4965,  0.5678,
        -0.6210,  0.2357,  0.2500,  0.1224,  0.6855, -0.4967, -0.2727,  0.5628,
        -0.4314, -0.3739,  0.4154, -0.5793], grad_fn=<SelectBackward>)

In [12]:
hidden1[1]

tensor([-0.2189,  0.4443,  0.1174,  0.0303,  0.1716, -0.0728, -0.4512,  0.1068,
        -0.5840, -0.2048,  0.3376,  0.3189,  0.8684, -0.2691, -0.2673,  0.5449,
        -0.3267,  0.2016,  0.5241, -0.3732], grad_fn=<SelectBackward>)

nn.ReLU
^^^^^^^^^^^^^^^^^^^^^^
Non-linear activations are what create the complex mappings between the model's inputs and outputs.
They are applied after linear transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use `nn.ReLU <https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html>`_ between our
linear layers, but there's other activations to introduce non-linearity in your model.



In [13]:
#Non-linear activations are what create the complex mappings 
#between the model's inputs and outputs. They are applied 
#after linear transformations to introduce nonlinearity,
#helping neural networks learn a wide variety of phenomena

print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.4893,  0.2240,  0.3864,  0.4155,  0.2082, -0.3094, -0.4965,  0.5678,
         -0.6210,  0.2357,  0.2500,  0.1224,  0.6855, -0.4967, -0.2727,  0.5628,
         -0.4314, -0.3739,  0.4154, -0.5793],
        [-0.2189,  0.4443,  0.1174,  0.0303,  0.1716, -0.0728, -0.4512,  0.1068,
         -0.5840, -0.2048,  0.3376,  0.3189,  0.8684, -0.2691, -0.2673,  0.5449,
         -0.3267,  0.2016,  0.5241, -0.3732],
        [-0.4037,  0.2799, -0.0401,  0.4938,  0.2600, -0.0719, -0.5386,  0.3710,
         -0.5940, -0.1001,  0.4364,  0.2349,  0.7623, -0.3035,  0.0297,  0.2282,
         -0.4089, -0.0583,  0.1635, -0.5100]], grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0000, 0.2240, 0.3864, 0.4155, 0.2082, 0.0000, 0.0000, 0.5678, 0.0000,
         0.2357, 0.2500, 0.1224, 0.6855, 0.0000, 0.0000, 0.5628, 0.0000, 0.0000,
         0.4154, 0.0000],
        [0.0000, 0.4443, 0.1174, 0.0303, 0.1716, 0.0000, 0.0000, 0.1068, 0.0000,
         0.0000, 0.3376, 0.3189, 0.8684, 0.0000, 0.000

nn.Sequential
^^^^^^^^^^^^^^^^^^^^^^
`nn.Sequential <https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html>`_ is an ordered 
container of modules. The data is passed through all the modules in the same order as defined. You can use
sequential containers to put together a quick network like ``seq_modules``.



In [14]:
# nn.Sequential is a box which consists of different kinds of
# sushi in a pre-defined order. You can eat them only in that specific order! 
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28) # 3 images of size W and H.
logits = seq_modules(input_image) # forward pass
print(logits)

tensor([[ 0.2018, -0.2114, -0.1312,  0.1975,  0.0179,  0.2415,  0.2323, -0.0155,
         -0.0540,  0.1613],
        [ 0.3213, -0.3414, -0.0272,  0.2551, -0.0499,  0.2508,  0.2478, -0.0540,
          0.0624,  0.0278],
        [ 0.1488, -0.2254, -0.2278,  0.0778,  0.0174,  0.2283,  0.1619,  0.0282,
         -0.1122,  0.1134]], grad_fn=<AddmmBackward>)


nn.Softmax
^^^^^^^^^^^^^^^^^^^^^^
The last linear layer of the neural network returns `logits` - raw values in [-\infty, \infty] - which are passed to the
`nn.Softmax <https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html>`_ module. The logits are scaled to values 
[0, 1] representing the model's predicted probabilities for each class. ``dim`` parameter indicates the dimension along 
which the values must sum to 1. 



In [15]:
softmax = nn.Softmax(dim=1) # get PROBABILITY scores, scaled to [0, 1] by softmax operation
pred_probab = softmax(logits)

In [16]:
pred_probab

tensor([[0.1134, 0.0750, 0.0813, 0.1129, 0.0944, 0.1180, 0.1169, 0.0913, 0.0878,
         0.1089],
        [0.1264, 0.0651, 0.0892, 0.1183, 0.0872, 0.1178, 0.1174, 0.0868, 0.0976,
         0.0942],
        [0.1124, 0.0773, 0.0771, 0.1047, 0.0985, 0.1217, 0.1138, 0.0996, 0.0865,
         0.1085]], grad_fn=<SoftmaxBackward>)

Model Parameters
-------------------------
Many layers inside a neural network are *parameterized*, i.e. have associated weights 
and biases that are optimized during training. Subclassing ``nn.Module`` automatically 
tracks all fields defined inside your model object, and makes all parameters 
accessible using your model's ``parameters()`` or ``named_parameters()`` methods. 

In this example, we iterate over each parameter, and print its size and a preview of its values. 




In [17]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0080,  0.0324, -0.0334,  ..., -0.0068,  0.0246,  0.0051],
        [ 0.0292, -0.0339, -0.0269,  ..., -0.0058, -0.0018, -0.0237]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0078, -0.0131], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0340, -0.0068,  0.0200,  ...,  0.0346, -0.0175,  0.0385],
        [ 0.0212, -0.0300,  0.0084,  ..., -0.0147,  0.0057, -0.0077]],
       device='cuda

--------------




Further Reading
--------------
- `torch.nn API <https://pytorch.org/docs/stable/nn.html>`_

